In [1]:
import sys
sys.path.insert(0, "..\\utility")

In [2]:
#import initialize_tf
import tensorflow as tf
from data_loading import *
from data_preprocessing import *
from plotting_helpers import *
import tensorflow as tf
import matplotlib as mpl
from matplotlib import pyplot
import numpy as np
%matplotlib inline
mpl.rcParams['figure.figsize'] = [10, 10]
mpl.rcParams['figure.dpi'] = 72

In [3]:
training_data_name = "fd=0.3-0.7_hdf5"
testing_data = "califa"

In [4]:
x_dataset_orig, y_dataset_orig, metadata_orig = load_hdf5_data(name=training_data_name, count=None)
num_classes = 2 # get from unique
print(x_dataset_orig.shape)
print(y_dataset_orig.shape)


E:\Anaconda3\envs\honours\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is a Python keyword: 'class'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


(54000, 100, 100)
(54000, 100, 100)
(54000,)


In [ ]:
plot_classification_results(images=x_dataset_orig, y_preds=y_dataset_orig, y_trues=y_dataset_orig, y_labels=("No Disc", "Disc"), random_sample=True)

In [ ]:
x_dataset = x_dataset_orig.copy()
y_dataset = y_dataset_orig.copy()
metadata = metadata_orig.copy()

In [ ]:
add_point_sources(x_dataset)

In [ ]:
add_background_galaxies(x_dataset) 

In [ ]:
apply_gaussian_PSF(x_dataset, 3)

In [ ]:
add_gaussian_noise(x_dataset)

In [ ]:
plot_classification_results(images=x_dataset, y_preds=y_dataset, y_trues=y_dataset, y_labels=("No Disc", "Disc"), random_sample=True)

In [ ]:
x_dataset = np.expand_dims(x_dataset, axis=3)
input_shape = x_dataset[0].shape
print(input_shape)

In [ ]:
def train_model(x_dataset, y_dataset):

    from sklearn import model_selection
    x_val, Y_val, metadata = load_hdf5_data(name=testing_data)
    x_val = np.expand_dims(x_val, axis=3)
    #Y_val = tf.keras.utils.to_categorical(Y_val, num_classes)
    print(x_val.shape)

    x_train, x_test, y_train, y_test = model_selection.train_test_split(x_dataset, y_dataset, train_size=0.8, random_state=1)

    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

    epochs = 20
    batch_size = 64
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                        activation='relu',
                        input_shape=input_shape),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(x_dataset.shape[0], activation='softmax')
    ])

    model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                    optimizer=tf.keras.optimizers.Adadelta(learning_rate=1.0),
                    metrics=['accuracy'])

    train_history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
        verbose=1, validation_data=(x_val, Y_val), callbacks=[callback])
    pyplot.plot(train_history.history['loss'])
    pyplot.plot(train_history.history['val_loss'])
    pyplot.legend(['loss', 'val_loss'])
    pyplot.show()

In [ ]:
#for epoch in range(epochs):
#      history = model.fit(x_train, y_train, batch_size=batch_size, epochs=1,
#            verbose=1, validation_data=(x_test, y_test))
#      loss = history.history['loss']
      #score = model.evaluate(x_test, y_test, verbose=0)
      #print('Test score:', score[0])
      #print('Test accuracy:', score[1])

In [ ]:
model.save(r"saved_models\simulation_psf_" + str(psf_sigma), overwrite=True)